# Imaging-AMARETTO: an imaging genomics software tool to systematically interrogate multi-omics networks for relevance to radiography and histopathology imaging biomarkers of clinical outcomes with application to studies of brain tumors

#### Olivier Gevaert<sup>&#35;</sup>, Mohsen Nabian<sup>&#35;</sup>, Shaimaa Bakr, Celine Everaert, Jayendra Shinde, Artur Manukyan, Ted Liefeld, Thorin Tabor, Jishu Xu, Joachim Lupberger, Brian J. Haas, Thomas F. Baumert, Mikel Hernaez, Michael Reich, Francisco Quintana, Erik J. Uhlmann, Anna M. Krichevsky, Jill P. Mesirov<sup>&#42;</sup>, Vincent Carey<sup>&#42;</sup>, Nathalie Pochet<sup>&#42;</sup>



## The &#42;AMARETTO framework
We present <B>&#42;AMARETTO</B> as a software toolbox for network biology and medicine, towards developing a data-driven platform for diagnostic, prognostic and therapeutic decision-making in cancer. &#42;AMARETTO links diseases, drivers, targets and drugs via network graph-based fusion of multi-omics, clinical, imaging, and driver and drug perturbation data across model systems and patient studies of cancer. The &#42;AMARETTO platform features a modular approach to incorporating prior biological knowledge based on multimodal and multiscale network-structured modeling:

<B>1.</B> The <B>AMARETTO</B> algorithm learns networks of regulatory modules or circuits - circuits of drivers and target genes - from functional genomics or multi-omics data and associates these circuits to clinical, molecular and imaging-derived phenotypes within each biological system (e.g., model systems or patients);<BR>
<B>2.</B> The <B>Community-AMARETTO</B> algorithm learns subnetworks of regulatory circuits that are shared or distinct across networks derived from multiple biological systems (e.g., model systems and patients, cohorts and individuals, diseases and etiologies, in vitro and in vivo systems);<BR>
<B>3.</B> The <B>Perturbation-AMARETTO</B> algorithm maps genetic and chemical perturbations in model systems onto patient-derived networks for driver and drug discovery, respectively, and prioritizes lead drivers, targets and drugs for follow-up with experimental validation;<BR>
<B>4.</B> The <B>Imaging-AMARETTO</B> algorithm maps radiographic and histopathology imaging data onto the patient-derived multi-omics networks for non-invasive and histopathology imaging diagnostics.

## In this tutorial, we will guide you through the following steps for running the Imaging-AMARETTO toolbox in R via GitHub <br><br>

<B>Step 1</B>: Before you begin, install software for running Imaging-AMARETTO<br>

<B>Step 2</B>: Preparing data for running Imaging-AMARETTO<br>

<B>Step 3</B>: Running Imaging-AMARETTO to identify multi-omics regulatory networks for the TCGA GBM cohort<br>

<B>Step 4</B>: Running Imaging-AMARETTO to identify multi-omics regulatory networks for the TCGA LGG cohort<br>

<B>Step 5</B>: Running Imaging-AMARETTO to identify transcriptional regulatory networks for the IvyGAP GBM cohort<br>

<B>Step 6</B>: Running Imaging-Community-AMARETTO to identify regulatory subnetworks shared or distinct across the TCGA GBM, IvyGAP GBM and TCGA LGG cohorts<br>

<B>Time complexity</B>, <B>Resources</B>, <B>References</B>, and <B>Funding</B><br><br>

<B>Questions?</B>

For any questions with the Imaging-AMARETTO Notebooks, please contact <B>Nathalie Pochet</B> (<npochet@broadinstitute.org>) and <B>Olivier Gevaert</B> (<ogevaert@stanford.edu>).


#Before you begin, install software for running Imaging-AMARETTO

Import libraries required for running the Imaging-AMARETTO in R Notebook by running the following code cells.

The Imaging-AMARETTO toolbox can be installed from GitHub (development versions) or from Bioconductor (future: official releases).


### Installing systems libraries on Linux server

Installing libraries on Google Colaboratory or own Linux servers using the collowing code cell. Note: this requires sudo access.

In [ ]:
system("sudo apt-get install libv8-dev", intern = TRUE, ignore.stderr = TRUE)

In [ ]:
if (!requireNamespace("BiocManager", quietly = TRUE))
    install.packages("BiocManager")

In [ ]:
BiocManager::install("ComplexHeatmap")
install.packages("devtools")
library(devtools)

In [ ]:
if (!requireNamespace("BiocManager", quietly = TRUE))
  install.packages("BiocManager")

BiocManager::install("ivygapSE")
library(ivygapSE)

### Installing and loading Imaging-AMARETTO

Installing the latest versions of Imaging-AMARETTO. This may take a few minutes...  ( In case of getting installation error, you can skip the line and run the next installation commands. 

In [ ]:
devtools::install_github("broadinstitute/ImagingAMARETTO ", ref="master", dependencies=TRUE)

In [ ]:
library("ImagingAMARETTO")

installationNote : Only run the following command if you recieve an error for the previous two commands. Otherwise, simply skip the following command.

In [ ]:
devtools::install_github("gevaertlab/AMARETTO", ref="master", dependencies=TRUE)
library("AMARETTO")
devtools::install_github("broadinstitute/CommunityAMARETTO", ref="master", dependencies=TRUE)
library("CommunityAMARETTO")

#A. Running Imaging-AMARETTO to identify multi-omics or transcriptional regulatory networks for the TCGA GBM, IvyGAP GBM and TCGA LGG cohorts


##A1. Running Imaging-AMARETTO to identify multi-omics regulatory networks for the TCGA GBM cohort

### Loading multi-omics data from TCGA GBM cohort

Loading Multi-Omics data including Gene Expression (MA), Copy Number Variation (CNV) and DNA Methylation (MET) : (Only MA is mandatory) 

In [ ]:
MA_matrix_GBM <-read_gct(url('http://portals.broadinstitute.org/pochetlab/demo/NotebookExample/ExampleData/TCGA_GBM_Expression.gct'))
CNV_matrix_GBM <-read_gct(url('http://portals.broadinstitute.org/pochetlab/demo/NotebookExample/ExampleData/TCGA_GBM_CNV.gct'))
MET_matrix_GBM <-read_gct(url('http://portals.broadinstitute.org/pochetlab/demo/NotebookExample/ExampleData/TCGA_GBM_Methylation.gct'))

In [ ]:
ProcessedData_TCGA_GBM = list(MA_matrix=MA_matrix_GBM, CNV_matrix=CNV_matrix_GBM, MET_matrix=MET_matrix_GBM)

### Defining List(s) of Candidate Driver Genes  (Optional)

In case CNV and/or MET data are submitted in addition to MA data, you can choose to either (<B>1</B>) use those as candidate drivers, or (<B>2</B>) take the union of these with predefined lists of candidate drivers, or (<B>3</B>) take the intersection with predefined lists of candidate drivers.

Alternatively, for example, if only MA data is available, you should select or upload a predefined list of candidate driver genes (required).

In [ ]:
data(Driver_Genes)
candidate_drivers<-Driver_Genes$Cancer_Gene_Census

### Setting parameters for running Imaging-AMARETTO (Required)

Core parameters that can be set by the user for running Imaging-AMARETTO include:

(<B>1</B>) <B>Number of regulatory modules</B> (i.e., cell circuits and their drivers) to be inferred. As a rule of thumb, hiqh quality regulatory modules  comprise of ~60-80 genes, so the optimal range of number of modules can be calculated by dividing the total number of genes in the analysis (see parameter % variation) by these numbers. Depending on the number of genes in the analysis, the ideal suggested range is ~100-200 modules.

(<B>2</B>) <B>Percentage of most varying genes</B> across the sample population (% genes) to be included in the analysis. This is based on the functional genomics data, which can be population RNA-Seq, single-cell RNA-Seq, or proteomics data. While genes that do not vary across the population (i.e., stdev zero) are automatically filtered out from the analysis, it is recommended to adjust the % variation filter for each dataset. Depending on the type of data, the ideal suggested range is ~25%-75% genes that vary the most across the population.

In [ ]:
NrModules = 150
VarPercentage = 75

### Running Imaging-AMARETTO for inferring regulatory networks from multi-omics data

By running the following code cell, you can now run the Imaging-AMARETTO algorithm. This may take some time (see section "Time complexity of Imaging-AMARETTO" below).

In [ ]:
ProcessedData_TCGA_GBM = list(MA_matrix=MA_matrix_GBM, CNV_matrix=CNV_matrix_GBM, MET_matrix=MET_matrix_GBM)
AMARETTOinit_GBM <- AMARETTO_Initialize(ProcessedData = ProcessedData_TCGA_GBM,
                                        Driver_list = candidate_drivers,
                                        method = "union",
                                        NrModules = NrModules,
                                        VarPercentage = VarPercentage,
                                        NrCores = 4)

AMARETTOresults_GBM<-AMARETTO_Run(AMARETTOinit_GBM)

### Functional Enrichment Analysis (Optional)
We here Perform <B> Fuctional Enrichment Analysis</B> using a <B>collections of known functional categories</B> for functional characterization of the regulatory modules or cell circuits, such as, for example, the Hallmark and C2 collections from MSigDB, Immune signatures, Stromal signatures and stemess signatures. More collections can be downloaded from <http://software.broadinstitute.org/gsea/msigdb/collections.jsp>) and/or uploaded by the user in .GMT format.


In [ ]:
functional_gmts<-c(url("http://portals.broadinstitute.org/pochetlab/demo/NotebookExample/ExampleData/H_C2_genesets.gmt"),
                   url("http://portals.broadinstitute.org/pochetlab/demo/NotebookExample/ExampleData/ImmuneSignatures.gmt"),
                   url("http://portals.broadinstitute.org/pochetlab/demo/NotebookExample/ExampleData/StromalSignatures.gmt"),
                   url("http://portals.broadinstitute.org/pochetlab/demo/NotebookExample/ExampleData/StemnessSignatures.gmt"))
GBM_hgtest_tbl<-HyperGeoEnrichmentTest(AMARETTOresults=AMARETTOresults_GB,hyper_geo_reference=functional_gmts,driverGSEA=TRUE, NrCores=5)

### Driver Validation (Optional)
We here Perform <B> Driver Validation study </B> using a <B>collections of known driver perturbation studies</B> for the driver validation of the inferred Imaging-AMARETTO regulatory modules. 

In [ ]:
genetic_gmts<-c(url("http://portals.broadinstitute.org/pochetlab/demo/NotebookExample/ExampleData/GeneticPerturbationSignatures.gmt"),
                url("http://portals.broadinstitute.org/pochetlab/demo/NotebookExample/ExampleData/ChEABindingSignatures.gmt"),
                url("http://portals.broadinstitute.org/pochetlab/demo/NotebookExample/ExampleData/ConsensusBindingSignatures.gmt"),
                url("http://portals.broadinstitute.org/pochetlab/demo/NotebookExample/ExampleData/EncodeBindingSignatures.gmt"))
GBM_hgtest_tbl_gp<-HyperGeoEnrichmentTest(AMARETTOresults=AMARETTOresults_GBM,hyper_geo_reference=genetic_gmts,driverGSEA=TRUE, NrCores=5)

### Imaging and Clinical Phenotype Association 

(<B>2</B>) We here define 1) <B>Imaging phenotypes</B> as well as <B>Cilinical and Molecular phenotypes</B> and 2) their corresponding statistical tests to be implemented for phenotype association. Imaging and Clinical and Phenotypes are used for visualization on the heatmaps of regulatory modules generated by Imaging-AMARETTO, as well as tested for their associations with regulatory modules using statistical tests provided by Imaging-AMARETTO.

In [ ]:
sample_annotations_GBM<-readr::read_delim(url("http://portals.broadinstitute.org/pochetlab/demo/NotebookExample/ExampleData/Samples_GBM_phenotypes.txt"),delim="\t")
phenotype_tests_GBM<-readr::read_delim(url("http://portals.broadinstitute.org/pochetlab/demo/NotebookExample/ExampleData/Samples_GBM_statistics.txt"),delim="\t",col_names=TRUE)
GBM_phenotype_tests_all<-AMARETTO_unite_results(AMARETTO_statistical_test(AMARETTOinit_GBM,AMARETTOresults_GBM,sample_annotations_GBM_all,phenotype_tests_GBM_all))

### Generating an HTML report for Imaging-AMARETTO results
By running the following code cell, the Imaging-AMARETTO report will be generated and can be accessed in the left panel. To distingusish <B> Imaging Phenotypes </B> we provide appropriate keywords for imaging_phenotypes_keywords parameter. (here: "vasari")

In [ ]:
dir.create('./AMARETTO_results') 
dir.create('./AMARETTO_results/GBM_Report_GCO') 
pp<-AMARETTO_HTMLreport(AMARETTOinit=AMARETTOinit_GBM,
                    AMARETTOresults=AMARETTOresults_GBM,
                    ProcessedData_TCGA_GBM,
                    hyper_geo_reference=GBM_hgtest_tbl,
                    genetic_pert_hyper_geo_reference = GBM_hgtest_tbl_gp,
                    output_address='./AMARETTO_results/GBM_Report_GCO',
                    SAMPLE_annotation=sample_annotations_GBM,
                    ID='Sample',
                    show_row_names=FALSE,
                    phenotype_association_table = GBM_phenotype_tests_all,
                    imaging_phenotypes_keywords = c("vasari"))

##A2. Running Imaging-AMARETTO to identify multi-omics regulatory networks for the TCGA LGG cohort


Loading Multi-Omics data including Gene Expression (MA), Copy Number Variation (CNV) and DNA Methylation (MET) : (Only MA is mandatory) 

In [ ]:
MA_matrix_LGG <-read_gct(url('http://portals.broadinstitute.org/pochetlab/demo/NotebookExample/ExampleData/TCGA_LGG_Expression.gct'))
CNV_matrix_LGG <-read_gct(url('http://portals.broadinstitute.org/pochetlab/demo/NotebookExample/ExampleData/TCGA_LGG_CNV.gct'))
MET_matrix_LGG <-read_gct(url('http://portals.broadinstitute.org/pochetlab/demo/NotebookExample/ExampleData/TCGA_LGG_Methylation.gct'))

In [ ]:
ProcessedData_TCGA_LGG = list(MA_matrix=MA_matrix_LGG, CNV_matrix=CNV_matrix_LGG, MET_matrix=MET_matrix_LGG)

### Defining List(s) of Candidate Driver Genes  (Optional)

In case CNV and/or MET data are submitted in addition to MA data, you can choose to either (<B>1</B>) use those as candidate drivers, or (<B>2</B>) take the union of these with predefined lists of candidate drivers, or (<B>3</B>) take the intersection with predefined lists of candidate drivers.

Alternatively, for example, if only MA data is available, you should select or upload a predefined list of candidate driver genes (required).

In [ ]:
data(Driver_Genes)
candidate_drivers<-Driver_Genes$Cancer_Gene_Census

Running Imaging-AMARETTO core for regulatory network inference :
Number of regulatory modules (NrModules), percentage of most varying genes (VarPercentage) are required. Here we specified NrModules and VarPercentage to be 150 and 75 respectively. We can optionally specify number of cores (NrCores) for parallel processing. As for the combination method for (1) the computed and (2) the predefined list of drivers, we specified "union" (as opposed to "intersection").


### Setting parameters for running Imaging-AMARETTO (Required)

Core parameters that can be set by the user for running Imaging-AMARETTO include:

(<B>1</B>) <B>Number of regulatory modules</B> (i.e., cell circuits and their drivers) to be inferred. As a rule of thumb, hiqh quality regulatory modules  comprise of ~60-80 genes, so the optimal range of number of modules can be calculated by dividing the total number of genes in the analysis (see parameter % variation) by these numbers. Depending on the number of genes in the analysis, the ideal suggested range is ~100-200 modules.

(<B>2</B>) <B>Percentage of most varying genes</B> across the sample population (% genes) to be included in the analysis. This is based on the functional genomics data, which can be population RNA-Seq, single-cell RNA-Seq, or proteomics data. While genes that do not vary across the population (i.e., stdev zero) are automatically filtered out from the analysis, it is recommended to adjust the % variation filter for each dataset. Depending on the type of data, the ideal suggested range is ~25%-75% genes that vary the most across the population.

In [ ]:
NrModules = 150
VarPercentage = 75

### Running Imaging-AMARETTO for inferring regulatory networks from multi-omics data

By running the following code cell, you can now run the Imaging-AMARETTO algorithm. This may take some time (see section "Time complexity of Imaging-AMARETTO" below).

In [ ]:
AMARETTOinit_LGG <- AMARETTO_Initialize(ProcessedData = ProcessedData_TCGA_LGG,
                                         Driver_list = all_drives_LGG,
                                         method = "union",
                                         NrModules = NrModules,
                                         VarPercentage = VarPercentage,
                                         NrCores = 4)
AMARETTOresults_LGG<-AMARETTO_Run(AMARETTOinit_LGG)

### Functional Enrichment Analysis (Optional)
We here Perform <B> Fuctional Enrichment Analysis</B> using a <B>collections of known functional categories</B> for functional characterization of the regulatory modules or cell circuits, such as, for example, the Hallmark and C2 collections from MSigDB, Immune signatures, Stromal signatures and stemess signatures. More collections can be downloaded from <http://software.broadinstitute.org/gsea/msigdb/collections.jsp>) and/or uploaded by the user in .GMT format.


In [ ]:
functional_gmts<-c(url("http://portals.broadinstitute.org/pochetlab/demo/NotebookExample/ExampleData/H_C2_genesets.gmt"),
                   url("http://portals.broadinstitute.org/pochetlab/demo/NotebookExample/ExampleData/ImmuneSignatures.gmt"),
                   url("http://portals.broadinstitute.org/pochetlab/demo/NotebookExample/ExampleData/StromalSignatures.gmt"),
                   url("http://portals.broadinstitute.org/pochetlab/demo/NotebookExample/ExampleData/StemnessSignatures.gmt"))
LGG_hgtest_tbl<-HyperGeoEnrichmentTest(AMARETTOresults=AMARETTOresults_LGG,hyper_geo_reference=functional_gmts,driverGSEA=TRUE, NrCores=5)

### Driver Validation (Optional)
We here Perform <B> Driver Validation study </B> using a <B>collections of known driver perturbation studies</B> for the driver validation of the inferred Imaging-AMARETTO regulatory modules. 

In [ ]:
genetic_gmts<-c(url("http://portals.broadinstitute.org/pochetlab/demo/NotebookExample/ExampleData/GeneticPerturbationSignatures.gmt"),
                url("http://portals.broadinstitute.org/pochetlab/demo/NotebookExample/ExampleData/ChEABindingSignatures.gmt"),
                url("http://portals.broadinstitute.org/pochetlab/demo/NotebookExample/ExampleData/ConsensusBindingSignatures.gmt"),
                url("http://portals.broadinstitute.org/pochetlab/demo/NotebookExample/ExampleData/EncodeBindingSignatures.gmt"))
LGG_hgtest_tbl_gp<-HyperGeoEnrichmentTest(AMARETTOresults=AMARETTOresults_LGG,hyper_geo_reference=genetic_gmts,driverGSEA=TRUE, NrCores=5)

### Imaging and Clinical Phenotype Association (Optional)

(<B>2</B>) We here define 1) <B>Imaging phenotypes</B> as well as <B>Clinical and Molecular phenotypes</B> and 2) their corresponding statistical tests to be implemented for phenotype association. Imaging and Clinical and Phenotypes are used for visualization on the heatmaps of regulatory modules generated by Imaging-AMARETTO, as well as tested for their associations with regulatory modules using statistical tests provided by Imaging-AMARETTO.

In [ ]:
sample_annotations_LGG_all<-read.csv(url("http://portals.broadinstitute.org/pochetlab/demo/NotebookExample/ExampleData/sample_annotations_LGG_all.csv"))
LGG_phenotype_tests_LGG_all<-read.csv(url("http://portals.broadinstitute.org/pochetlab/demo/NotebookExample/ExampleData/phenotype_tests_LGG_all.csv"))
LGG_phenotype_tests_all<-AMARETTO_unite_results(AMARETTO_statistical_test(AMARETTOinit_LGG,AMARETTOresults_LGG,sample_annotations_LGG_all,phenotype_tests_LGG_all))

### Generating an HTML report for Imaging-AMARETTO results

By running the following code cell, the Imaging-AMARETTO report will be generated and can be accessed in the left panel. To distingusish <B> Imaging Phenotypes </B> we provide appropriate keywords for imaging_phenotypes_keywords parameter (here: "vasari").

In [ ]:
dir.create('./AMARETTO_results/LGG_Report_GCO') 
pp<-AMARETTO_HTMLreport(AMARETTOinit=AMARETTOinit_LGG,
                    AMARETTOresults=AMARETTOresults_LGG,
                    ProcessedData_TCGA_LGG,
                    hyper_geo_reference=LGG_hgtest_tbl,
                    genetic_pert_hyper_geo_reference =LGG_hgtest_tbl_gp,
                    output_address='./AMARETTO_results/LGG_Report_GCO',
                    SAMPLE_annotation=sample_annotations_LGG_all,
                    ID='Sample',
                    show_row_names=FALSE,
                    phenotype_association_table = LGG_phenotype_tests_all,
                    imaging_phenotypes_keywords = c("vasari"))

##A3. Running Imaging-AMARETTO to identify transcriptional regulatory networks for the IvyGAP GBM cohort


Loading Gene Expression (MA) data for IvyGAP GBM : We do not use Copy Number Variation (CNV) and DNA Methylation (MET) in this analysis. 

In [ ]:
MA_matrix_ivy_normalized1<-read_gct(url('http://portals.broadinstitute.org/pochetlab/demo/NotebookExample/ExampleData/MA_matrix_ivy_normalized1.gct'))
ProcessedData_Ivygap_GBM = list(MA_matrix=MA_matrix_ivy_normalized1, CNV_matrix=NULL, MET_matrix=NULL)

### Defining List(s) of Candidate Driver Genes  (Required)

In case CNV and/or MET data are submitted in addition to MA data, you can choose to either (<B>1</B>) use those as candidate drivers, or (<B>2</B>) take the union of these with predefined lists of candidate drivers, or (<B>3</B>) take the intersection with predefined lists of candidate drivers.

Alternatively, for example, if only MA data is available, you should select or upload a predefined list of candidate driver genes (required).

In [ ]:
data(Driver_Genes)
candidate_drivers<-Driver_Genes$Cancer_Gene_Census

Running Imaging-AMARETTO core for regulatory network inference :
Number of regulatory modules (NrModules), percentage of most varying genes (VarPercentage) are required. Here we specified NrModules and VarPercentage to be 150 and 75 respectively. We can optionally specify number of cores (NrCores) for parallel processing. As for the combination method for (1) the computed and (2) the predefined list of drivers, we specified "union" (as opposed to "intersection").


### Setting parameters for running Imaging-AMARETTO (Required)

Core parameters that can be set by the user for running Imaging-AMARETTO include:

(<B>1</B>) <B>Number of regulatory modules</B> (i.e., cell circuits and their drivers) to be inferred. As a rule of thumb, hiqh quality regulatory modules  comprise of ~60-80 genes, so the optimal range of number of modules can be calculated by dividing the total number of genes in the analysis (see parameter % variation) by these numbers. Depending on the number of genes in the analysis, the ideal suggested range is ~100-200 modules.

(<B>2</B>) <B>Percentage of most varying genes</B> across the sample population (% genes) to be included in the analysis. This is based on the functional genomics data, which can be population RNA-Seq, single-cell RNA-Seq, or proteomics data. While genes that do not vary across the population (i.e., stdev zero) are automatically filtered out from the analysis, it is recommended to adjust the % variation filter for each dataset. Depending on the type of data, the ideal suggested range is ~25%-75% genes that vary the most across the population.

In [ ]:
NrModules = 150
VarPercentage = 50

### Running Imaging-AMARETTO for inferring regulatory networks from multi-omics data

By running the following code cell, you can now run the Imaging-AMARETTO algorithm. This may take some time (see section "Time complexity of Imaging-AMARETTO" below).

In [ ]:
AMARETTOinit_ivy <- AMARETTO_Initialize(ProcessedData = ProcessedData_TCGA_ivy,
                                        Driver_list = all_drives,
                                        method = "union",
                                        NrModules = NrModules,
                                        VarPercentage = VarPercentage,
                                        NrCores = 8)

AMARETTOresults_ivy<-AMARETTO_Run(AMARETTOinit_ivy)

### Functional Enrichment Analysis (Optional)
We here Perform <B> Fuctional Enrichment Analysis</B> using a <B>collections of known functional categories</B> for functional characterization of the regulatory modules or cell circuits, such as, for example, the Hallmark and C2 collections from MSigDB, Immune signatures, Stromal signatures and stemess signatures. More collections can be downloaded from <http://software.broadinstitute.org/gsea/msigdb/collections.jsp>) and/or uploaded by the user in .GMT format.


In [ ]:
functional_gmts<-c(url("http://portals.broadinstitute.org/pochetlab/demo/NotebookExample/ExampleData/H_C2_genesets.gmt"),
                   url("http://portals.broadinstitute.org/pochetlab/demo/NotebookExample/ExampleData/ImmuneSignatures.gmt"),
                   url("http://portals.broadinstitute.org/pochetlab/demo/NotebookExample/ExampleData/StromalSignatures.gmt"),
                   url("http://portals.broadinstitute.org/pochetlab/demo/NotebookExample/ExampleData/StemnessSignatures.gmt"))
ivy_hgtest_tbl<-HyperGeoEnrichmentTest(AMARETTOresults=AMARETTOresults_ivy,hyper_geo_reference=functional_gmts,driverGSEA=TRUE, NrCores=5)

### Driver Validation (Optional)
We here Perform <B> Driver Validation study </B> using a <B>collections of known driver perturbation studies</B> for the driver validation of the inferred Imaging-AMARETTO regulatory modules. 

In [ ]:
genetic_gmts<-c(url("http://portals.broadinstitute.org/pochetlab/demo/NotebookExample/ExampleData/GeneticPerturbationSignatures.gmt"),
                url("http://portals.broadinstitute.org/pochetlab/demo/NotebookExample/ExampleData/ChEABindingSignatures.gmt"),
                url("http://portals.broadinstitute.org/pochetlab/demo/NotebookExample/ExampleData/ConsensusBindingSignatures.gmt"),
                url("http://portals.broadinstitute.org/pochetlab/demo/NotebookExample/ExampleData/EncodeBindingSignatures.gmt"))
ivy_hgtest_tbl_gp<-HyperGeoEnrichmentTest(AMARETTOresults=AMARETTOresults_ivy,hyper_geo_reference=genetic_gmts,driverGSEA=TRUE, NrCores=5)

### Imaging and Clinical Phenotype Association (Optional)

(<B>2</B>) We here define 1) <B>Imaging phenotypes</B> as well as <B>Clinical and Molecular phenotypes</B> and 2) their corresponding statistical tests to be implemented for phenotype association. Imaging and Clinical and Phenotypes are used for visualization on the heatmaps of regulatory modules generated by Imaging-AMARETTO, as well as tested for their associations with regulatory modules using statistical tests provided by Imaging-AMARETTO.

In [ ]:
sample_annotations_ivy_all<-read.csv(url("http://portals.broadinstitute.org/pochetlab/demo/NotebookExample/ExampleData/sample_annotations_LGG_all.csv"))
phenotype_tests_ivy_all<-read.csv(url("http://portals.broadinstitute.org/pochetlab/demo/NotebookExample/ExampleData/phenotype_tests_LGG_all.csv"))
ivy_phenotype_tests_all<-AMARETTO_unite_results(AMARETTO_statistical_test(AMARETTOinit_ivy,AMARETTOresults_ivy,sample_annotations_ivy_all,phenotype_tests_ivy_all))

### Generating an HTML report for Imaging-AMARETTO results

By running the following code cell, the Imaging-AMARETTO report will be generated and can be accessed in the left panel. To distingusish <B> Imaging Phenotypes </B> we provide appropriate keywords for imaging_phenotypes_keywords parameter . (here : "anatomic_" and "cancer_" )

In [ ]:
dir.create('./AMARETTO_results/ivy_Report_GCO') 
AMARETTO_HTMLreport(AMARETTOinit=AMARETTOinit_ivy,
                    AMARETTOresults=AMARETTOresults_ivy,
                    ProcessedData_Ivygap_GBM,
                    hyper_geo_reference=ivy_hgtest_tbl,
                    genetic_pert_hyper_geo_reference = ivy_hgtest_tbl_gp,
                    output_address='./AMARETTO_results/ivy_Report_GCO',
                    SAMPLE_annotation=sample_annotations_ivy_all,
                    ID='Sample',
                    show_row_names=FALSE,
                    phenotype_association_table = ivy_phenotype_tests_all,
                    imaging_phenotypes_keywords = c("anatomic_","cancer_"))

# B. Running Imaging-Community-AMARETTO to identify regulatory subnetworks shared or distinct across the TCGA GBM, IvyGAP GBM and TCGA LGG cohorts

The Imaging-Community-AMARETTO algorithm takes as input results from two or more previous Imaging-AMARETTO analyses to identify regulatory subnetworks or communities (i.e., cell circuits and their drivers) that are shared and distinct across multiple datasets, cohorts, biological systems and diseases.

##Preparing data and parameter settings for running Imaging-Community-AMARETTO

### Create a list of upstream Imaging-AMARETTO results files :

We here define a list "AMARETTOresults_all", containing Imaging-AMARETTO results files for the Imaging-Community-AMARETTO analysis. 


In [ ]:
AMARETTOresults_all<-list(TCGA_GBM=AMARETTOresults_GBM,
                                  TCGA_LGG=AMARETTOresults_LGG,
                                  Ivygap_GBM = AMARETTOresults_ivy)


### Create a list from all phenotype association studies  :
We define a list "PhenotypeTables" that contain all the phenotype tables being calculated for individual datasets. 

In [ ]:
PhenotypeTables<-list(TCGA_GBM=GBM_phenotype_tests_all,
                      TCGA_LGG=LGG_phenotype_tests_all,
                      Ivygap_GBM =ivy_phenotype_tests_all)


### Create a vector containing links to upstream Imaging-AMARETTO html reports for all datasets  :

Provide a vector containing links to upstream Imaging-AMARETTO html reports for all datasets. These links are being used for creating hyperlinks to individual Imaging-AMARETTO module pages inside the Imaging-Community-AMARETTO report.

In [ ]:
HTMLsAMARETTOlist <- c("TCGA_GBM"="./AMARETTO_results/GBM_Report_GCO",
                       "TCGA_LGG"="./AMARETTO_results/LGG_Report_GCO",
                       "Ivygap_GBM"='./AMARETTO_results/ivy_Report_GCO')

### Loading additional networks as a set of signatures in .GMT format (Optional)

One or more additional networks can be submitted as signatures files in .GMT format and combined by running the following code cell. These networks are used in Imaging-Community-AMARETTO as separate networks. In this example, we submit previously published signatures and/or networks, such as immune signatures from CiberSort and stemness signatures from <i>Ben-Porath et al.</i> as two independent networks to be analyzed.

If additional networks are submitted, please run following cell code to include them in the analysis.

In [ ]:
ImmuneSignatures <- "ImmuneSignatures.gmt"
download.file(url="https://www.broadinstitute.org/~npochet/NotebookExample/ExampleData/ImmuneSignatures.gmt", destfile=ImmuneSignatures)

StemSignatures <- "StemSignatures.gmt"
download.file(url="https://www.broadinstitute.org/~npochet/NotebookExample/ExampleData/StemSignatures.gmt", destfile=StemSignatures)


list_additional_networks = list(ImmuneSignatures = "ImmuneSignatures.gmt", StemSignatures = "StemSignatures.gmt", LiverSignatures = "LiverSignatures.gmt")

Otherwise set to NULL.

In [ ]:
list_additional_networks = NULL

### Setting parameters for computing on servers (Optional)

Selecting the number of cores for parallel computing.

In [ ]:
NrCores = 4

### Running Imaging-Community-AMARETTO

By running the following code cells, you can now run the Imaging-Community-AMARETTO algorithm. This may take some time (see section "Time complexity of Imaging-AMARETTO" below).

You can run the following code cell for computing the extent of overlaps between all regulatory modules learned across data cohorts.

In [ ]:
cAMARETTOresults<-cAMARETTO_Results(AMARETTOresults_all = AMARETTOresults_all,
                                    gmt_filelist = list_additional_networks,
                                    NrCores = NrCores,
                                    drivers = FALSE)

Creating and filtering edges in the module networks across data cohorts. Additional parameters that can be set by the user in this step include:

(<B>1</B>) <B>Filtering edges in the subnetworks or communities based on p-value significance</B>. Edges in the network with p-values larger than the cutoff value will be filtered out. The default cutoff p-value is 0.05.<br>

(<B>2</B>) <B>Filtering edges in the subnetworks or communities based on the minimum number overlapping genes</B>. Edges in the network with the number of overlapping genes less than the cutoff value will be filtered out. The default cutoff value is 5 overlapping genes.

In [ ]:
cAMARETTOnetworkM<-cAMARETTO_ModuleNetwork(cAMARETTOresults,
                                           pvalue = 0.05,
                                           inter = 5)

Identifying and filtering edges in the subnetworks across data cohorts. Additional parameters that can be set by the user in this step include:

<B>Filtering edges in the subnetworks or communities that do not satisfy all the following conditions</B>:<br>
(<B>1</B>) Number of nodes in the community larger than the 1% of the total number of nodes in the network,<br>
(<B>2</B>) Number of represented datasets/cohorts in the community larger than the 10% of the subnetwork size (and at least, larger than 2),<br>
(<B>3</B>) Ratio between edges inside/outside the community larger than 0.5.<br>

The user can choose between filtering according to these criteria, in which case edges in the network that do not satisfy all of these criteria will be filtered out, or whether to not apply these filtering criteria to retain all edges.

In [ ]:
cAMARETTOnetworkC<-cAMARETTO_IdentifyCom(cAMARETTOnetworkM,
                                         filterComm = FALSE,
                                         ratioCommSize = 0.01,
                                         MinRuns = 2,
                                         ratioRunSize = 0.1,
                                         ratioEdgesInOut = 0.5)

### Saving and downloading Imaging-Community-AMARETTO results files and HTML report

By running the following code cell, you can download a .zip file of the Imaging-Community-AMARETTO report as well as .rds data files in the left panel by right-clicking on each file and selecting *Download*. 

In [ ]:
saveRDS(cAMARETTOresults, file="cAMARETTOresults.rds")
saveRDS(cAMARETTOnetworkM, file="cAMARETTOnetworkM.rds")
saveRDS(cAMARETTOnetworkC, file="cAMARETTOnetworkC.rds")

### Functional Enrichment Analysis (Optional)
We here Perform <B> Fuctional Enrichment Analysis</B> using a <B>collections of known functional categories</B> for functional characterization of the regulatory modules or cell circuits, such as, for example, the Hallmark and C2 collections from MSigDB, Immune signatures, Stromal signatures and stemess signatures. More collections can be downloaded from <http://software.broadinstitute.org/gsea/msigdb/collections.jsp>) and/or uploaded by the user in .GMT format.


In [ ]:
functional_gmts<-c(url("http://portals.broadinstitute.org/pochetlab/demo/NotebookExample/ExampleData/H_C2_genesets.gmt"),
                   url("http://portals.broadinstitute.org/pochetlab/demo/NotebookExample/ExampleData/ImmuneSignatures.gmt"),
                   url("http://portals.broadinstitute.org/pochetlab/demo/NotebookExample/ExampleData/StromalSignatures.gmt"),
                   url("http://portals.broadinstitute.org/pochetlab/demo/NotebookExample/ExampleData/StemnessSignatures.gmt"))
cAMARETTO_hgtest_tbl<-CreateHyperGeoTestAll(cAMARETTOresults,cAMARETTOnetworkM,cAMARETTOnetworkC,hyper_geo_reference=functional_gmts,driverGSEA=TRUE,NrCores=5)

### Driver Validation (Optional)
We here Perform <B> Driver Validation study </B> using a <B>collections of known driver perturbation studies</B> for the driver validation inferred communities. 

In [ ]:
genetic_gmts<-c(url("http://portals.broadinstitute.org/pochetlab/demo/NotebookExample/ExampleData/GeneticPerturbationSignatures.gmt"),
                url("http://portals.broadinstitute.org/pochetlab/demo/NotebookExample/ExampleData/ChEABindingSignatures.gmt"),
                url("http://portals.broadinstitute.org/pochetlab/demo/NotebookExample/ExampleData/ConsensusBindingSignatures.gmt"),
                url("http://portals.broadinstitute.org/pochetlab/demo/NotebookExample/ExampleData/EncodeBindingSignatures.gmt"))
cAMARETTO_hgtest_tbl_gp<-CreateHyperGeoTestAll(cAMARETTOresults,cAMARETTOnetworkM,cAMARETTOnetworkC,hyper_geo_reference=genetic_gmts,driverGSEA=TRUE,NrCores=5)

## Generating Imaging-Community-AMARETTO HTML report
Finally, We here generate the Imaging-Community-AMARETTO results as html reports. 

In [ ]:
dir.create('./cAmarettoRep/GBM_Ivygap_GCO') 
cAMARETTO_HTMLreport(cAMARETTOresults,
                     cAMARETTOnetworkM, 
                     cAMARETTOnetworkC,
                     HTMLsAMARETTOlist = HTMLsAMARETTOlist,
                     CopyAMARETTOReport = TRUE, 
                     hyper_geo_reference = cAMARETTO_hgtest_tbl,
                     hyper_geo_reference_gp = cAMARETTO_hgtest_tbl_gp,
                     output_address = "./cAmarettoRep/GBM_Ivygap_GCO",
                     PhenotypeTablesList = PhenotypeTables)
                    

In [ ]:
zip(zipfile="cAMARETTO_report.zip",files="./cAmarettoRep/GBM_Ivygap_GCO")

##  Viewing Imaging-Community-AMARETTO results combining multiple Imaging-AMARETTO analyses

The .zip file generated in previous step provides a queryable .html Imaging-Community-AMARETTO report that directly links to the original Imaging-AMARETTO analyses from which they were derived. The <B>Imaging-Community-AMARETTO report</B> includes:

(<B>1</B>) An overview <B>index.html</B> page that provides:
- A summary of the analyzed networks (including links to the original Imaging-AMARETTO networks derived from multiple datasets)
- A network graph overview of the subnetworks or communities learned across multiple datasets
- A queryable overview table of shared/distinct communities with assignments of modules to communities across all datasets
- A queryable overview table of shared/distinct drivers of communities with assignments of drivers to communities across all datasets
- A queryable overview table of gene assignments (drivers and targets) to communities
- A queryable overview table with enrichments of known functional categories in communities

(<B>2</B>) For each regulatory subnetwork or community a <B>community#.html</B> page is generated that provides:
- A network graph visualization of the subnetworks or communities learned across multiple datasets
- A queryable table of shared/distinct communities across multiple datasets, including regulatory modules that are shared/distinct across datasets
- A queryable table of gene assignments (drivers and targets) in communities
- A queryable table with enrichments of known functional categories in communities

Overview and community-specific Tables and Network Graphs can be saved as .CSV, .Excel, .PDF and .PNG files.

# Time complexity of Imaging-AMARETTO

Depending on the size of the data, for example, for TCGA cohorts of ~300-500 samples, it can take up to several hours to run the Imaging-AMARETTO algorithms and generate reports on the Google Colab servers.

# Source code, user-friendly tools and resources

The source code, user-friendly tools and resources of the current &#42;AMARETTO toolbox and future developments are available from GitHub, Bioconductor, GenePattern, GenomeSpace, GenePattern Notebook and R Jupyter Notebook, as updated here: <http://portals.broadinstitute.org/pochetlab/amaretto.html>.

Imaging-AMARETTO resources are available from: <http://portals.broadinstitute.org/pochetlab/JCO_CCI_Imaging-AMARETTO/Imaging-AMARETTO_HTML_Report_TCGA-GBM_IVYGAP-GBM_TCGA-LGG/index.html> and the HTML report generated by this R Juputer Notebook is here: <http://portals.broadinstitute.org/pochetlab/JCO_CCI_Imaging-AMARETTO/Imaging-AMARETTO_HTML_Report_TCGA-GBM_IVYGAP-GBM_TCGA-LGG/index.html>.



# References

1. Multiscale and multimodal inference of regulatory networks using &#42;AMARETTO. <i>In preparation for submission</i>.

2. Champion M., Brennan K., Croonenborghs T., Gentles A.J., Pochet N., Gevaert O. (2018). Module Analysis Captures Pancancer Genetically and Epigenetically Deregulated Cancer Driver Genes for Smoking and Antiviral Response. <i>EBioMedicine</i>, 27:156-166. PMID:29331675 PMCID:PMC5828545.

3. Gevaert O., Villalobos V., Sikic B.I., Plevritis S.K. (2014). Identification of ovarian cancer driver genes by using module network integration of multi-omics data. <i>Interface Focus</i>, 3(4):20130013. PMID:24511378 PMCID:PMC3915833.

4. Manolakos A., Ochoa I., Venkat K., Goldsmith A.J., Gevaert O. (2014). CaMoDi: a new method for cancer module discovery. <i>BMC Genomics</i>, 15 Suppl 10:S8. PMID:25560933 PMCID:PMC4304219.

5. Gevaert O., Plevritis S. (2013). Identifying master regulators of cancer and their downstream targets by integrating genomic and epigenomic features. <i>Pacific Symposium on Biocomputing</i>, 2013:123-34. PMID:23424118 PMCID:PMC3911770.

6. Cedoz P.L., Prunello M., Brennan K., Gevaert O. (2018). MethylMix 2.0: an R package for identifying DNA methylation genes. <i>Bioinformatics</i>, 34(17):3044-3046. PMID:29668835 PMCID:PMC6129298.

7. Gevaert O., Tibshirani R., Plevritis S.K. (2015). Pancancer analysis of DNA methylation-driven genes using MethylMix. <i>Genome Biology</i>, 16(1):17. PMID:25631659 PMCID:PMC4365533.

8. Gevaert O. (2015). MethylMix: an R package for identifying DNA methylation-driven genes. <i>Bioinformatics</i>, 31(11):1839-41. PMID:25609794 PMCID:PMC4443673.

9. Stubbs B.J., Gopaulakrishnan S., Glass K., Pochet N., Everaert C., Raby B., Carey V. (2019). TFutils: Data structures for transcription factor bioinformatics. <i>F1000Research</i>, 8:152. (<https://doi.org/10.12688/f1000research.17976.1>).

10. Qu K., Garamszegi S., Wu F., Thorvaldsdottir H., Liefeld T., Ocana M., Borges-Rivera D., Pochet N., Robinson J.T., Demchak B., Hull T., Ben-Artzi G., Blankenberg D., Barber G.P., Lee B.T., Kuhn R.M., Nekrutenko A., Segal E., Ideker T., Reich M., Regev A., Chang H.Y., Mesirov J.P. (2016). Integrative genomic analysis by interoperation of bioinformatics tools in GenomeSpace. <i>Nature Methods</i>, 13(3):245-247. PMID:26780094 PMCID:PMC4767623.

11. Reich M., Liefeld T., Ocana M., Jang D., Bistline J., Robinson J., Carr P., Hill B., McLaughlin J., Pochet N., Borges-Rivera D., Tabor T., Thorvaldsdottir H., Regev A., Mesirov J.P. (2013). GenomeSpace: an environment for frictionless bioinformatics. <i>F1000Posters</i>, 4:804. (<https://f1000research.com/posters/1093972>).





# Funding

This work was supported by grants from NIH NCI ITCR R21 CA209940 (Pochet), NIH NCI ITCR U01 CA214846 Collaborative Supplement (Carey/Pochet) and NIH NIAID R03 AI131066 (Pochet).

# Questions?

For any questions with the &#42;AMARETTO Notebooks, please contact <B>Nathalie Pochet</B> (<npochet@broadinstitute.org>) and <B>Olivier Gevaert</B> (<ogevaert@stanford.edu>).